<a href="https://colab.research.google.com/github/AliAkbarBadri/graph-centrality/blob/main/axiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Prerequisite**

In [1]:
import networkx as nx
import pandas as pd
import glob
import time
import numpy as np
from random import *


In [2]:

def count_subgraph(G, v):
  neighbours = G.edges(v)
  if len(neighbours) == 0:
    return 1
  else:
    for e in neighbours:
      G_temp = G.copy()
      G_temp.remove_edge(*e)
      a = count_subgraph(G_temp, v)
      b = (2**G.number_of_edges(*e) - 1)*count_subgraph(nx.contracted_edge(G, e, self_loops=False), e[0])
      return a + b
def count_tree(T, v):
  neighbours = T.edges(v)
  if len(neighbours) == 0:
    return 1
  else:
    for e in neighbours:
      T_temp = T.copy()
      T_temp.remove_edge(*e)
      return count_tree(T_temp, v) * (count_tree(T_temp,e[1]) + 1)

! curl -L https://api.github.com/repos/AliAkbarBadri/graph-centrality/tarball --output repo.tar
! tar xf repo.tar --wildcards "*/data/*.csv" --strip-components=1
! rm -rf repo.tar

def create_graph_from_csv(dir_name:str) -> dict:
  graphs = {}
  for file_name in sorted(glob.glob(dir_name+'/*.csv')):
    df = pd.read_csv(file_name,header=None,names=['src', 'dest'])
    G = nx.from_pandas_edgelist(df,source="src",target="dest", create_using=nx.DiGraph())
    dataset_name = file_name.split("/")[1].split(".csv")[0]
    print(dataset_name, G.number_of_nodes(), G.number_of_edges())
    graphs[dataset_name] = G
  return graphs

graphs = create_graph_from_csv(dir_name="data")

  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  457k    0  457k    0     0  1103k      0 --:--:-- --:--:-- --:--:-- 1103k
ENZYMES_g297 121 298
aves-songbird-social 110 1027
bn-mouse_visual-cortex_1 29 44
bn-mouse_visual-cortex_2 193 214
ca-sandi_auths 86 124
email-dnc-corecipient 906 12085
email-enron-only 143 623
facebook-socfb-Amherst41 2235 90953
insecta-ant-colony1-day01 113 4550
reptilia-tortoise-network-lm 45 106
road-chesapeake 39 170
rt-retweet 96 117
soc-dolphins 62 159
soc-wiki-Vote 889 2914
web-edu 3031 6474
web-polblogs 643 2280
web-spam 4767 37375


# **AXIOM 1**

axiom 1 :monotonicity : tree centrality

In [ ]:
def check_monotonicity_tree(G, number_node):
    # axiom 1 : monotonicity
    # test C(v,G-e) <= C(v,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = choice(list(G.nodes()))
        removed_edge = choice(list(G.edges()))
        score_v = count_tree(G, random_node)
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_tree(G_temp, random_node)
        check = score_v_without <= score_v
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for node " + str(random_node) + " , remove " + str(
        #     removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate tree centrality =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


axiom 1 :monotonicity : subgraph centrality

In [ ]:
def check_monotonicity_subgraph(G, number_node):
    # axiom 1 : monotonicity
    # test C(v,G-e) <= C(v,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = choice(list(G.nodes()))
        removed_edge = choice(list(G.edges()))
        score_v = count_subgraph(G, random_node)
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_tree(G_temp, random_node)
        check = score_v_without <= score_v
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for node " + str(random_node) + " , remove " + str(
            # removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate subgraph centrality =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


test axiom 1

In [ ]:
number_node = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<50 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("-----------")
    check_monotonicity_tree(G,number_node)
    check_monotonicity_subgraph(G,number_node)
    print("------------------------")


Graph: bn-mouse_visual-cortex_1
-----------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
Graph: reptilia-tortoise-network-lm
-----------
Success rate tree centrality =  100%
Success rate subgraph centrality =  20%
------------------------
Graph: road-chesapeake
-----------
Success rate tree centrality =  100%


# **AXIOM 2**

axiom 2 : rank monotonicity : tree centrality

In [ ]:
def check_rank_monotonicity_tree(G, number_node):
    # test if C(v,G-e)<= C(u,G-e) then C(v,G) <= C(u,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = sample(list(G.nodes()),2)
        removed_edge = choice(list(G.edges()))
        score_v = count_tree(G, random_node[0])
        score_u = count_tree(G, random_node[1])
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_tree(G_temp, random_node[0])
        score_u_without = count_tree(G_temp, random_node[1])
        check = (score_v_without <= score_u_without and score_v <= score_u) or (score_u_without <= score_v_without and score_u <= score_v)
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for nodes " + str(random_node) + " , remove " + str(
        #     removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate tree centrality =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


axiom 2 : rank monotonicity : subgraph centrality

In [ ]:
def check_rank_monotonicity_subgraph(G, number_node):
    # test if C(v,G-e)<= C(u,G-e) then C(v,G) <= C(u,G) for 10 random node
    check_array = [0, 0]
    for i in range(number_node):
        random_node = sample(list(G.nodes()),2)
        removed_edge = choice(list(G.edges()))
        score_v = count_subgraph(G, random_node[0])
        score_u = count_subgraph(G, random_node[1])
        G_temp = G.copy()
        G_temp.remove_edge(*removed_edge)
        score_v_without = count_subgraph(G_temp, random_node[0])
        score_u_without = count_subgraph(G_temp, random_node[1])
        check = (score_v_without <= score_u_without and score_v <= score_u) or (score_u_without <= score_v_without and score_u <= score_v)
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for nodes " + str(random_node) + " , remove " + str(
        #     removed_edge) + " edge , meet condition? " + str(check))
    print("Success rate subgraph centrality =  " + str(round((check_array[0] / (check_array[0] + check_array[1])) * 100)) + "%")


test axiom 2

In [ ]:
number_node = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<50 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("-------------")
    check_rank_monotonicity_tree(G,number_node)
    check_rank_monotonicity_subgraph(G,number_node)
    print("------------------------")


Graph: bn-mouse_visual-cortex_1
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
Graph: reptilia-tortoise-network-lm
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
Graph: road-chesapeake
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------


# **AXIOM 3**

axiom 3 : line minimization : tree centrality

In [ ]:
def check_line_minimization_tree(G, number_node):
    # test C(0,Ln) <= C(v,G) for 10 random node
    check_array = [0, 0]
    G.remove_nodes_from(list(nx.isolates(G)))
    for i in range(number_node):
        random_node = choice(list(G.nodes()))
        score_v = count_tree(G, random_node)
        line = nx.path_graph(G.number_of_nodes())
        score_line = count_tree(line,0)
        check = score_line <= score_v
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for node " + str(random_node) +" meet condition? " + str(check))
    rate = round((check_array[0] / (check_array[0] + check_array[1])) * 100)
    print("Success rate tree centrality =  " + str(rate) + "%")
    return rate


axiom 3 : line minimization : subgraph centrality

In [ ]:
def check_line_minimization_subgraph(G, number_node):
    # test C(0,Ln) <= C(v,G) for 10 random node
    check_array = [0, 0]
    G.remove_nodes_from(list(nx.isolates(G)))
    for i in range(number_node):
        random_node = choice(list(G.nodes()))
        score_v = count_subgraph(G, random_node)
        line = nx.path_graph(G.number_of_nodes())
        score_line = count_subgraph(line,0)
        check = score_line <= score_v
        if check:
            check_array[0] += 1
        else:
            check_array[1] += 1
        # print(str(i) + "st round : compute score for node " + str(random_node) +" meet condition? " + str(check))
    rate = round((check_array[0] / (check_array[0] + check_array[1])) * 100)
    print("Success rate subgraph centrality =  " + str(rate) + "%")
    return rate

test axiom 3

In [ ]:
number_node = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<50 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("-------------")
    check_line_minimization_tree(G,number_node)
    check_line_minimization_subgraph(G,number_node)
    print("------------------------")


Graph: bn-mouse_visual-cortex_1
-------------
Success rate tree centrality =  20%
Success rate subgraph centrality =  50%
------------------------
Graph: reptilia-tortoise-network-lm
-------------
Success rate tree centrality =  40%
Success rate subgraph centrality =  30%
------------------------
Graph: road-chesapeake
-------------
Success rate tree centrality =  80%


# **AXIOM 4**

axiom 4 : continuity : tree centrality

In [3]:
def check_continuity_tree(v,H,G):
  check_array = [0, 0]
  check = False
  G1 = H
  edges = G.edges()
  list_edges = []
  G1_edges = G1.edges()
  edges = edges - G1_edges
  tree_G = count_tree(G,v)
  for edge in edges :
    G1_temp= G1
    G1_temp.add_edge(edge)
    tree_G1 = count_tree(G1,v)
    tree_G1_temp = count_tree(G1_temp,v)
    if tree_G1 < tree_G1_temp :
      list_edges.append(edge)
      G1 = G1_temp
      if tree_G1_temp == tree_G :
        print ("find sequence edge!", list_edges)
        check_array[0]+=1
        check = True
  if check == False : check_array[1]+=1
  return check_array




axiom 4 : continuity : subgraph centrality

In [7]:
G = graphs['bn-mouse_visual-cortex_1']
G_temp = G
check_array = [0, 0]
for k in range(G.number_of_nodes()):
  nodes = []
  for i in range(G.number_of_nodes()):
    if i < k :
      nodes.append(i+1)
  G1 = G.subgraph(nodes)
  for i in range(1,G1.number_of_nodes()):
    if (count_tree(G1,i)<count_tree(G,i)): 
      print("condition match")
      G1_temp = G1
      check_array += check_continuity_tree(i,G1_temp,G_temp)
rate = round((check_array[0] / (check_array[0] + check_array[1])) * 100)
print("Success rate tree centrality =  " + str(rate) + "%")

condition match


NetworkXError: ignored

# **Axiom 5**

axiom 5 : size : tree centrality

In [ ]:
def check_size_tree(G, number_n):
    # test N <= C(0,G) for 10 random N 
    check_array = [0, 0]
    check = False
    G.remove_nodes_from(list(nx.isolates(G)))
    for i in range(number_n):
        random_n = randint(0,G.number_of_edges())
        for node in list(G.nodes()):
          score_v = count_tree(G, node)
          if score_v > random_n : check = True
          if check : break
        if check : check_array[0]+=1
        else : check_array[1]+=1
        # print(str(i) + "st round : compute score for n " + str(random_n) +" meet condition? " + str(check))
    rate = round((check_array[0] / (check_array[0] + check_array[1])) * 100)
    print("Success rate tree centrality =  " + str(rate) + "%")
    return rate

axiom 5 : size : subgraph centrality

In [ ]:
def check_size_subgraph(G, number_n):
    # test N <= C(0,G) for 10 random N 
    check_array = [0, 0]
    check = False
    G.remove_nodes_from(list(nx.isolates(G)))
    for i in range(number_n):
        random_n = randint(0,G.number_of_edges())
        for node in list(G.nodes()):
          score_v = count_subgraph(G, node)
          if score_v > random_n : check = True
          if check : break
        if check : check_array[0]+=1
        else : check_array[1]+=1
        # print(str(i) + "st round : compute score for n " + str(random_n) +" meet condition? " + str(check))
    rate = round((check_array[0] / (check_array[0] + check_array[1])) * 100)
    print("Success rate subgraph centrality =  " + str(rate) + "%")
    return rate

test axiom 5

In [ ]:
number_n = 10
for key, value in graphs.items():
  if graphs[key].number_of_nodes()<50 :
    G = graphs[key]
    print("Graph: {}".format(key))
    print("-------------")
    check_size_tree(G,number_n)
    check_size_subgraph(G,number_node)
    print("------------------------")


Graph: bn-mouse_visual-cortex_1
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
Graph: reptilia-tortoise-network-lm
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
Graph: road-chesapeake
-------------
Success rate tree centrality =  100%
Success rate subgraph centrality =  100%
------------------------
